In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_parquet("ratings.parquet")
df.head()

,noteId,ratedOnTweetId,raterParticipantId,noteAuthorParticipantId,postCreatedAt,noteCreatedAt,ratingCreatedAt,helpfulnessLevel,fromNotification
0,1357294499052085249,1357269755112148993,FE4BA8F630E66459C9F2EEBDDF69C4B22D222AD55E9013...,FB8A5B50F04AAB7CCEA99B4094EAA1AD8E49E3503EF18E...,2021-02-04 10:08:21.128,2021-02-04 11:46:40.544,2026-02-01 14:36:13.314,NOT_HELPFUL,False
1,1357297935407603713,1357269755112148993,FE4BA8F630E66459C9F2EEBDDF69C4B22D222AD55E9013...,B9BEE8138A8A67538391BC79C1E9DFF614FCEE2DD089CB...,2021-02-04 10:08:21.128,2021-02-04 12:00:19.835,2026-02-01 14:36:04.345,NOT_HELPFUL,False
2,1358604958673719296,1358464064322691078,456D1C422BA1A09690B986C8A1DBE7627D10462AF965A0...,FAB1117EE31D648BA7C11062F626AF7B6908D551B4C9C8...,2021-02-07 17:14:06.633,2021-02-08 02:33:58.465,2026-02-09 08:56:34.712,NOT_HELPFUL,False
3,1358604958673719296,1358464064322691078,4B3821FC937ACD882F8BD4045DD281556154AC41E45387...,FAB1117EE31D648BA7C11062F626AF7B6908D551B4C9C8...,2021-02-07 17:14:06.633,2021-02-08 02:33:58.465,2026-02-09 00:51:34.502,NOT_HELPFUL,False
4,1358844963216326659,1358464064322691078,4B3821FC937ACD882F8BD4045DD281556154AC41E45387...,605A405C64CC57A943D086CC80E5AC35A57605B4CEE3D8...,2021-02-07 17:14:06.633,2021-02-08 18:27:40.007,2026-02-09 00:51:18.076,NOT_HELPFUL,False


In [3]:
# Time Transform
df["ratingCreatedAt"] = pd.to_datetime(df["ratingCreatedAt"])
df["noteCreatedAt"] = pd.to_datetime(df["noteCreatedAt"])
df["postCreatedAt"] = pd.to_datetime(df["postCreatedAt"])

df = df.sort_values(["raterParticipantId", "ratingCreatedAt"]).reset_index(drop=True)

In [4]:
# Was the rating apart of a "rating session,"
gap_prev = df.groupby("raterParticipantId")["ratingCreatedAt"].diff()
gap_next = df.groupby("raterParticipantId")["ratingCreatedAt"].diff(-1).abs()

df["is_rating_session"] = (
    (gap_prev <= pd.Timedelta(minutes=5)) |
    (gap_next <= pd.Timedelta(minutes=5))
)

df["is_rating_session"] = df["is_rating_session"].fillna(False)

df

,noteId,ratedOnTweetId,raterParticipantId,noteAuthorParticipantId,postCreatedAt,noteCreatedAt,ratingCreatedAt,helpfulnessLevel,fromNotification,is_rating_session
0,2016534713742061788,2016246745739546987,00002C7FD6E0080A69D0AB879C3D9BB704BEFCC078AD40...,FD7979DE7F187DC14A4FDF8F5572424D060AA56E50AB30...,2026-01-27 20:27:38.894,2026-01-28 15:31:55.816,2026-01-28 20:47:22.321,HELPFUL,False,False
1,2017641230612451495,2017578816177111423,00002C7FD6E0080A69D0AB879C3D9BB704BEFCC078AD40...,2CE75633DDBFF074EE77E7DC582D7B557DC4D737E5FD81...,2026-01-31 12:40:49.235,2026-01-31 16:48:49.996,2026-01-31 17:21:23.365,HELPFUL,False,False
2,2018404224149717157,2018387895459983768,00002C7FD6E0080A69D0AB879C3D9BB704BEFCC078AD40...,54F0002E8A4890D64140C45F5D108C9D76FDA829A157A0...,2026-02-02 18:15:48.768,2026-02-02 19:20:41.831,2026-02-02 22:44:01.605,HELPFUL,False,False
3,2012648760455671942,2012586343008755726,00003D7D222733AE9D37A25B9302040575F6157FECDDB6...,B5A7C39C8965D9D8AF5CC38B994AAC58DE5374B7660F88...,2026-01-17 18:02:30.876,2026-01-17 22:10:32.357,2026-01-18 15:41:27.835,NOT_HELPFUL,False,True
4,2012693431319990371,2012586343008755726,00003D7D222733AE9D37A25B9302040575F6157FECDDB6...,D174DF64F45A03029780F273B4335CE8D87C68A9DAF7EB...,2026-01-17 18:02:30.876,2026-01-18 01:08:02.720,2026-01-18 15:44:04.360,HELPFUL,False,True
...,...,...,...,...,...,...,...,...,...,...
6499630,2020752418963669036,2020604630657302841,FFFFBBAB3C66ABB4DBC2A3B486C3C673345C89B5858465...,BA48B1306E38D71468370309229E1842BDB92CF4F7917A...,2026-02-08 21:04:19.624,2026-02-09 06:51:35.102,2026-02-09 08:02:06.384,NOT_HELPFUL,False,False
6499631,2021916824426701208,2021784932818067904,FFFFBBAB3C66ABB4DBC2A3B486C3C673345C89B5858465...,E6ABEA01CD00795F7897A1245141FF57C05A9C136F6AE6...,2026-02-12 03:14:25.588,2026-02-12 11:58:30.999,2026-02-12 15:50:43.969,HELPFUL,False,False
6499632,2019488960465031197,2019467286298472468,FFFFC46B8555A97065DB39F7D600C8BB643F7F3EBD810E...,8D8A300EC055D96FC685C886688785A6A5F8BD49663F0F...,2026-02-05 17:44:55.598,2026-02-05 19:11:03.122,2026-02-06 08:34:11.398,HELPFUL,False,False
6499633,2013458308682690872,2013361050352050642,FFFFFE8909485374E33854B934713713CAC93CDB50C9D0...,4F0866C36C86735E6D9A3DF709775EEDCFA2180684FD72...,2026-01-19 21:20:55.500,2026-01-20 03:47:23.693,2026-01-20 22:31:31.668,HELPFUL,False,False


In [5]:
# Was the rating based on interest in the post
df["post_rating_count_by_user"] = (
    df.groupby(["raterParticipantId", "ratedOnTweetId"])
      ["ratedOnTweetId"]
      .transform("size")
)

df["is_interest_based"] = df["post_rating_count_by_user"] >= 2

df

,noteId,ratedOnTweetId,raterParticipantId,noteAuthorParticipantId,postCreatedAt,noteCreatedAt,ratingCreatedAt,helpfulnessLevel,fromNotification,is_rating_session,post_rating_count_by_user,is_interest_based
0,2016534713742061788,2016246745739546987,00002C7FD6E0080A69D0AB879C3D9BB704BEFCC078AD40...,FD7979DE7F187DC14A4FDF8F5572424D060AA56E50AB30...,2026-01-27 20:27:38.894,2026-01-28 15:31:55.816,2026-01-28 20:47:22.321,HELPFUL,False,False,1,False
1,2017641230612451495,2017578816177111423,00002C7FD6E0080A69D0AB879C3D9BB704BEFCC078AD40...,2CE75633DDBFF074EE77E7DC582D7B557DC4D737E5FD81...,2026-01-31 12:40:49.235,2026-01-31 16:48:49.996,2026-01-31 17:21:23.365,HELPFUL,False,False,1,False
2,2018404224149717157,2018387895459983768,00002C7FD6E0080A69D0AB879C3D9BB704BEFCC078AD40...,54F0002E8A4890D64140C45F5D108C9D76FDA829A157A0...,2026-02-02 18:15:48.768,2026-02-02 19:20:41.831,2026-02-02 22:44:01.605,HELPFUL,False,False,1,False
3,2012648760455671942,2012586343008755726,00003D7D222733AE9D37A25B9302040575F6157FECDDB6...,B5A7C39C8965D9D8AF5CC38B994AAC58DE5374B7660F88...,2026-01-17 18:02:30.876,2026-01-17 22:10:32.357,2026-01-18 15:41:27.835,NOT_HELPFUL,False,True,2,True
4,2012693431319990371,2012586343008755726,00003D7D222733AE9D37A25B9302040575F6157FECDDB6...,D174DF64F45A03029780F273B4335CE8D87C68A9DAF7EB...,2026-01-17 18:02:30.876,2026-01-18 01:08:02.720,2026-01-18 15:44:04.360,HELPFUL,False,True,2,True
...,...,...,...,...,...,...,...,...,...,...,...,...
6499630,2020752418963669036,2020604630657302841,FFFFBBAB3C66ABB4DBC2A3B486C3C673345C89B5858465...,BA48B1306E38D71468370309229E1842BDB92CF4F7917A...,2026-02-08 21:04:19.624,2026-02-09 06:51:35.102,2026-02-09 08:02:06.384,NOT_HELPFUL,False,False,1,False
6499631,2021916824426701208,2021784932818067904,FFFFBBAB3C66ABB4DBC2A3B486C3C673345C89B5858465...,E6ABEA01CD00795F7897A1245141FF57C05A9C136F6AE6...,2026-02-12 03:14:25.588,2026-02-12 11:58:30.999,2026-02-12 15:50:43.969,HELPFUL,False,False,1,False
6499632,2019488960465031197,2019467286298472468,FFFFC46B8555A97065DB39F7D600C8BB643F7F3EBD810E...,8D8A300EC055D96FC685C886688785A6A5F8BD49663F0F...,2026-02-05 17:44:55.598,2026-02-05 19:11:03.122,2026-02-06 08:34:11.398,HELPFUL,False,False,1,False
6499633,2013458308682690872,2013361050352050642,FFFFFE8909485374E33854B934713713CAC93CDB50C9D0...,4F0866C36C86735E6D9A3DF709775EEDCFA2180684FD72...,2026-01-19 21:20:55.500,2026-01-20 03:47:23.693,2026-01-20 22:31:31.668,HELPFUL,False,False,1,False


In [6]:
# Was the rating from a notification
df["is_notification"] = df["fromNotification"]

In [7]:
# Was the rating apart of a "rater swarm"

df = df.sort_values(["noteId", "ratingCreatedAt"]).reset_index(drop=True)

swarm_flag = {}
swarm_share = {}

for note_id, g in df.groupby("noteId"):
    times = g["ratingCreatedAt"].values
    n = len(times)

    if n < 5: 
        swarm_flag[note_id] = False
        swarm_share[note_id] = 0
        continue

    i = 0
    best = 1

    for j in range(n):
        while times[j] - times[i] > pd.Timedelta(hours=1):
            i += 1
        best = max(best, j - i + 1)

    share = best / n
    swarm_share[note_id] = share
    swarm_flag[note_id] = share >= 0.5

df["swarm_peak_share_1h"] = df["noteId"].map(swarm_share)
df["is_rater_swarm"] = df["noteId"].map(swarm_flag)

In [8]:
df[[
"is_rating_session",
"is_interest_based",
"is_notification",
"is_rater_swarm"
]].mean()

is_rating_session    0.590433
is_interest_based    0.509207
is_notification      0.019807
is_rater_swarm       0.033096
dtype: float64

The results show that approximately 59.0% of ratings occurred within a rating session, defined as multiple ratings by the same user within a five-minute window. This suggests that a majority of rating activity happens in batches, with many users evaluating several notes consecutively rather than rating only a single note in isolation.

About 50.9% of ratings were interest-based, meaning that the same user rated multiple notes associated with the same post. This indicates that a substantial portion of rating behavior may stem from engagement with a specific tweet, where users explore and evaluate multiple notes attached to it.

Only 2.0% of ratings originated from notifications. This suggests that notifications are not a major driver of rating activity, and most ratings appear to be user-initiated rather than prompted by system alerts.

3.3% of ratings were identified as part of a rater swarm, where a majority of ratings for a given note occurred within a short time window. This indicates that concentrated bursts of collective rating activity are relatively uncommon, and most notes receive ratings in a more distributed manner over time.